# Extreme value analysis

This module provides a wrapper for the Extremes.jl package, enabling its integration with Xarray. The Extremes.jl package allows the analysis of extreme values, providing the following features:

- Methods for block maxima and threshold exceedances (e.g., genextreme, gumbel_r, genpareto).
- Parameter estimation using probability-weighted moments, maximum likelihood, and Bayesian procedures.
- Stationary and non-stationary models.
- Return level estimation.

For more information about the Extremes.jl package, see: https://doi.org/10.18637/jss.v109.i06 and https://github.com/jojal5/Extremes.jl

In [ ]:
from pathlib import Path
from zipfile import ZipFile

import holoviews as hv
import hvplot.xarray  # noqa
import numpy as np
import pandas as pd
import panel as pn
import pooch
import xarray as xr

from xhydro.extreme_value_analysis.parameterestimation import fit, return_level
from xhydro.testing.helpers import deveraux

### Data

This example uses GFDL ESM4 annual total precipitation data from 1955 to 2100 for 97 virtual stations in Quebec.

In [ ]:
path_zip = deveraux().fetch("extreme_value_analysis/NOAA_GFDL_ESM4.zip")

directory_to_extract = Path(
    path_zip
).parent  # To extract to the same directory as the zip file
with ZipFile(path_zip, "r") as zip_ref:
    zip_ref.extractall(directory_to_extract)

path_csv = str(directory_to_extract) + "/data_NOAA_GFDL_ESM4.csv"

df = pd.read_csv(path_csv, parse_dates=[0])[
    ["time", "station_num", "station_name", "total_precip"]
]
ds = df.to_xarray()
ds = ds.set_coords(["time", "station_num", "station_name"])
ds = ds.set_index(index=["station_num", "time"])
ds = ds.unstack("index")

In [ ]:
station_sel = pn.widgets.Select(
    name="Select Station", options=ds.coords["station_num"].values.tolist()
)


def plot_station_data(station_num):
    station_data = ds.total_precip.sel(station_num=station_num)
    plot = station_data.hvplot.line(x="time", title=f"Virtual station {station_num}")
    return plot


interactive_plot = pn.bind(plot_station_data, station_num=station_sel)
pn.Column(station_sel, interactive_plot).servable()

### Parameter estimation for stationary model

GenExtreme parameter estimation is performed using maximum likelihood. Confidence intervals are calculated with 95% confidence.

In [ ]:
fit_stationary = fit(
    ds, dist="genextreme", method="ml", vars=["total_precip"], confidence_level=0.95
)
fit_stationary

### Return level estimation for stationary model

100-year return level for the Gumbel distribution using maximum likelihood. Confidence intervals are calculated with 95% confidence.

In [ ]:
rtnlv_stationary = return_level(
    ds,
    dist="gumbel_r",
    method="ml",
    vars=["total_precip"],
    confidence_level=0.95,
    return_period=100,
)
rtnlv_stationary

### Parameter estimation for non-stationary model

For this example the location paramerter vary as linear funcion of the year. To do this, a new dimension containing the year is created.

In [ ]:
ds2 = ds.copy(deep=True)
ds2["year"] = (
    ["station_num", "time"],
    np.tile(pd.to_datetime(ds.time).year, (ds2.dims["station_num"], 1)),
)
ds2

In [ ]:
fit_non_stationary = fit(
    ds2,
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    locationcov=["year"],
    confidence_level=0.95,
)
fit_non_stationary

### Return level estimation for non-stationary model

100-year return level with the location paramerter vary as linear funcion of the year.

In [ ]:
rtnlv_non_stationary = return_level(
    ds2,
    dist="genextreme",
    method="ml",
    vars=["total_precip"],
    locationcov=["year"],
    confidence_level=0.95,
    return_period=100,
)

rtnlv_non_stationary

# Comparison of the return level using the stationary and non-stationary model

In [ ]:
station_selector = pn.widgets.Select(
    name="Select Station",
    options=rtnlv_non_stationary.coords["station_num"].values.tolist(),
)


def plot_station_data(station_num):

    non_stationary = rtnlv_non_stationary.sel(station_num=station_num)
    stationary = rtnlv_stationary.sel(station_num=station_num)

    line_plot_sta = hv.Curve(
        (non_stationary.return_level.values, stationary.total_precip.values[0]),
        label="Stationary",
    ).opts(color="blue")
    area_plot = hv.Area(
        (
            list(non_stationary.return_level.values),
            stationary.total_precip_lower.values[0],
            stationary.total_precip_upper.values[0],
        ),
        vdims=["y", "y2"],
    ).opts(alpha=0.3, color="blue", line_width=0)

    area_plot_non = non_stationary.hvplot.area(
        x="return_level",
        y="total_precip_lower",
        y2="total_precip_upper",
        alpha=0.3,
        color="red",
        line_width=0,
    )
    line_plot_non = non_stationary["total_precip"].hvplot.line(
        x="return_level", label="Non-stationary", color="red"
    )

    plot = area_plot * line_plot_sta * area_plot_non * line_plot_non

    plot = plot.opts(
        title=f"Virtual Station {station_num}",
        xlabel="Return Level (Time)",
        ylabel="Total Precipitation",
    )

    return plot


interactive_plot = pn.bind(plot_station_data, station_num=station_selector)


pn.Column(station_selector, interactive_plot).servable()

# Working with dask.array chunks

Currently, the extreme_value_analysis function is not thread-safe. Therefore, it is recommended to use the dask.scheduler="processes" option to ensures that tasks are executed in separate Python processes.

In [ ]:
ds_c = ds.chunk({"time": -1, "station_num": 1})

fit_stationary_c = fit(
    ds_c, dist="genextreme", method="ml", vars=["total_precip"], confidence_level=0.95
)
fit_stationary_c

In [ ]:
fit_stationary_c.compute(scheduler="processes")